In [1]:
import warnings
warnings.filterwarnings('ignore')
import inspect
import pandas as pd
import numpy as np
from pydantic import BaseModel, EmailStr, AnyUrl, Field, field_validator, model_validator, computed_field
from typing import List, Dict, Optional, Annotated, Literal
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error
from xgboost import XGBRegressor
import json
import pickle
import joblib
import mlflow
from mlflow import MlflowClient

In [2]:
df = pd.read_csv('VN_housing_dataset.csv')
df = df.drop(columns=['Unnamed: 0'])
print(df.shape)
print(df.info())
df.head()

(82497, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82497 entries, 0 to 82496
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Ngày             82496 non-null  object
 1   Địa chỉ          82449 non-null  object
 2   Quận             82495 non-null  object
 3   Huyện            82449 non-null  object
 4   Loại hình nhà ở  82465 non-null  object
 5   Giấy tờ pháp lý  53610 non-null  object
 6   Số tầng          36399 non-null  object
 7   Số phòng ngủ     82458 non-null  object
 8   Diện tích        82495 non-null  object
 9   Dài              19827 non-null  object
 10  Rộng             35445 non-null  object
 11  Giá/m2           82484 non-null  object
dtypes: object(12)
memory usage: 7.6+ MB
None


,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m²,NaN,NaN,"86,96 triệu/m²"
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m²,NaN,NaN,"116,22 triệu/m²"
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m²,10 m,4 m,65 triệu/m²
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m²,12.75 m,4 m,100 triệu/m²
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m²,9 m,4 m,"86,11 triệu/m²"


In [3]:
df.columns = ['ngay', 'diachi', 'quan', 'huyen', 'loaihinhnhao', 'giaytophaply', 'sotang', 'sophongngu', 'dientich', 'dai', 'rong', 'dongia']
df = df[df.dongia.notna()]
print(df.shape[0])
df.head()

82484


,ngay,diachi,quan,huyen,loaihinhnhao,giaytophaply,sotang,sophongngu,dientich,dai,rong,dongia
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m²,NaN,NaN,"86,96 triệu/m²"
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m²,NaN,NaN,"116,22 triệu/m²"
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m²,10 m,4 m,65 triệu/m²
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m²,12.75 m,4 m,100 triệu/m²
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m²,9 m,4 m,"86,11 triệu/m²"


In [4]:
df = df[df['sotang'] != 'Nhiều hơn 10']
df['sotang'] = df['sotang'].fillna(0)
df = df[df['sophongngu'] != 'nhiều hơn 10 phòng']
print(df.shape[0])

81615


In [5]:
df['duong'] = df['diachi'].str.split(', ', expand=True)[0]
df['sotang'] = df['sotang'].astype(int)
df['sophongngu'] = df['sophongngu'].str.replace(' phòng','').str.strip().astype(float)
df['dientich'] = df['dientich'].str.split('m', expand=True)[0].astype(float)
df['dai'] = df['dai'].str.split('m', expand=True)[0].astype(float)
df['rong'] = df['rong'].str.split('m', expand=True)[0].astype(float)
print(df.shape[0] == 81615)
df.head()

True


,ngay,diachi,quan,huyen,loaihinhnhao,giaytophaply,sotang,sophongngu,dientich,dai,rong,dongia,duong
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,NaN,NaN,"86,96 triệu/m²",Đường Hoàng Quốc Việt
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,0,3.0,37.0,NaN,NaN,"116,22 triệu/m²",Đường Kim Giang
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4.0,40.0,10.00,4.0,65 triệu/m²,phố minh khai
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,0,6.0,51.0,12.75,4.0,100 triệu/m²,Đường Võng Thị
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,0,4.0,36.0,9.00,4.0,"86,11 triệu/m²",Đường Kim Giang


In [6]:
# df['dongia'] = df['dongia'].replace([','], ['.'])
# df

In [7]:
# Clean and convert all prices to million/m2 instead of VND/m2 or billion/m2
df.loc[df['dongia'].str.contains(' tỷ/m²'), 'dongia'] = df.loc[df['dongia'].str.contains(' tỷ/m²'), 'dongia'].str.replace(' tỷ/m²','').str.replace('.','').str.replace(',','.').astype(float) * 1000
df.loc[df['dongia'].str.contains(' triệu/m²', na=False), 'dongia'] = df.loc[df['dongia'].str.contains(' triệu/m²', na=False), 'dongia'].str.replace(' triệu/m²','').str.replace(',','.').astype(float)
df.loc[df['dongia'].str.contains(' đ/m²', na=False), 'dongia'] = df.loc[df['dongia'].str.contains(' đ/m²', na=False), 'dongia'].str.replace(' đ/m²','').str.replace('.','').astype(float) * 0.000001
# 4. Cuối cùng: chuyển toàn bộ cột sang float
df['dongia'] = pd.to_numeric(df['dongia'], errors='coerce')
df.head()

,ngay,diachi,quan,huyen,loaihinhnhao,giaytophaply,sotang,sophongngu,dientich,dai,rong,dongia,duong
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,NaN,NaN,86.96,Đường Hoàng Quốc Việt
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,0,3.0,37.0,NaN,NaN,116.22,Đường Kim Giang
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4.0,40.0,10.00,4.0,65.00,phố minh khai
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,0,6.0,51.0,12.75,4.0,100.00,Đường Võng Thị
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,0,4.0,36.0,9.00,4.0,86.11,Đường Kim Giang


In [8]:
# inter quantile range |---------Q1---------Q2-----------Q3------------|      ---
#                                 -------------------------   => IQR, any point < Q1 - 1.5*IQR, any point > Q3 + 1.5IQR  

## feature1, feature2, feature3, outlier cho tung feature, , neu row_index ma xay ra bat thuong it nhat n cot ==> nhieu kha nang
from collections import Counter

def detect_outliers(df,n,features):
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col],25)
        
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers fro feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index

        # append the found outlier indices for col to the list of outlier indices
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than n outliers
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(k for k,v in outlier_indices.items() if v > n)
    
    return multiple_outliers

In [9]:
def split_num_cat_cols(df, cols):
    cat_cols = []
    num_cols = []
    for col in df[cols].columns:
        if (col != 'ngay') & (col != 'dongia'):
            if df[col].dtype == 'O':
                cat_cols.append(col)
            else:
                num_cols.append(col)
    # print("Category Column:", cat_cols)
    # print("Numeric Column:", num_cols)
    return cat_cols, num_cols

In [10]:
def train_test(df, test_size):
    train, test = train_test_split(df, test_size = test_size, random_state = 42)

    return train, test

In [11]:
def perc_diff_transforms_less_than_10(x):
    if x < 10:
        return 1
    else:
        return 0

In [12]:
label = 'dongia'

In [13]:
def build_models(df, cols, test_size, params):

    cat_cols, num_cols = split_num_cat_cols(df, cols)
    features = cat_cols + num_cols
    # print(features)
    train, test = train_test(df = df, test_size = test_size)
    # print(len(train), len(test))

    outlier_to_drop = detect_outliers(train, 2, num_cols)
    # print("Number Outlier is:", len(outlier_to_drop))
    train = train.drop(outlier_to_drop, axis = 1)
    
    # Create column transformer for OHE
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown='ignore'), cat_cols),
            ("scale", StandardScaler(), num_cols)
        ]
    )

    # Create a pipeline with preprocessing and regression
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        # ("regressor", LinearRegression())
        ("regressor", XGBRegressor(**params))
    ])

    pipeline.fit(train[features], train[label])

    y_train_pred = pipeline.predict(train[features])
    y_test_pred = pipeline.predict(test[features])

    train['y_train_pred'] = y_train_pred
    train['perc_diff'] = abs((train['y_train_pred'] - train[label])) / train[label] * 100
    train['flag_perc_diff'] = train['perc_diff'].apply(perc_diff_transforms_less_than_10)
    # print(train['flag_perc_diff'].sum()/len(train))

    test['y_test_pred'] = y_test_pred
    test['perc_diff'] = abs((test['y_test_pred'] - test[label])) / test[label] * 100
    test['flag_perc_diff'] = test['perc_diff'].apply(perc_diff_transforms_less_than_10)
    # print(test['flag_perc_diff'].sum()/len(test))

    metrics = {
        'train_R2': r2_score(train['y_train_pred'], train[label]) * 100,
        'train_MAPE': mean_absolute_percentage_error(train['y_train_pred'], train[label]) * 100,
        'train_MAE': mean_absolute_error(train['y_train_pred'], train[label]) * 100,
        'train_accuracy' : train['flag_perc_diff'].sum()/len(train) * 100,
        'test_R2': r2_score(test['y_test_pred'], test[label]) * 100,
        'test_MAPE': mean_absolute_percentage_error(test['y_test_pred'], test[label]) * 100,
        'test_MAE': mean_absolute_error(test['y_test_pred'], test[label]) * 100,
        'test_accuracy' : test['flag_perc_diff'].sum()/len(test) * 100,
    }

    del train
    # print("Evaluation sucessfully!")

    evaluation_cols = ['y_test_pred', 'perc_diff', 'flag_perc_diff']
    
    return test[features + [label] + evaluation_cols], pipeline, metrics

In [14]:
params = {
        'objective': 'reg:squarederror', 
        'random_state': 42
}

In [15]:
# 1. Setup MLflow tracking & experiment
# =====================================================
TRACKING_URI = "file:///C:/Users/Admin/Documents/DataScience/DataOps/MLOps/modeling/real_state/mlflow"
EXPERIMENT_NAME = "real_state_experiment"
REGISTERED_MODEL_NAME = "RealStateModel"

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

client = MlflowClient()

2025/09/20 01:55:48 INFO mlflow.tracking.fluent: Experiment with name 'real_state_experiment' does not exist. Creating a new experiment.


In [16]:
# =====================================================
# 2. Train and log a model
# =====================================================
run_name = "real_state_model"

with mlflow.start_run(run_name=run_name) as run:
    for i in range(0, len(df.columns)):
        # print(df.columns[0:i+2])
        test, pipeline, metrics = build_models(df=df, cols=df.columns[0:i+2], test_size=0.2, params=params)
        # --- log hyperparameters & metrics
        mlflow.log_params(params)       # <-- your dict of params
        mlflow.log_metrics(metrics)     # <-- your dict of metrics

        # --- save the model as artifact of this run
        artifact_path = "model"
        mlflow.sklearn.log_model(
            sk_model=pipeline,
            name=artifact_path,
            input_example=test,
        )

In [17]:
# =====================================================
# 3. Search for the best run by metric (e.g. lowest rmse)
# =====================================================
runs_df = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    # order_by=["metrics.test_MAPE DESC"],      # sort DESC by MAPE
    order_by=["metrics.test_accuracy DESC"],      # sort DESC by MAPE
    max_results=1
)

best_run_id = runs_df.iloc[0].run_id
best_accuracy = runs_df.iloc[0]["metrics.test_accuracy"]
print(f"Best run: {best_run_id} (Accuracy = {best_accuracy})")

Best run: d831dcf83826454d841d41bc0d401950 (Accuracy = 27.917662194449548)


In [18]:
# =====================================================
# 4. Register that best run as a model
# =====================================================
model_uri = f"runs:/{best_run_id}/{artifact_path}"
registration = mlflow.register_model(model_uri, REGISTERED_MODEL_NAME)

Successfully registered model 'RealStateModel'.
2025/09/20 01:58:43 WARNING mlflow.tracking._model_registry.fluent: Run with id d831dcf83826454d841d41bc0d401950 has no artifacts at artifact path 'model', registering model based on models:/m-958126b2b1f64a6a9b042d4870ef87c6 instead
Created version '1' of model 'RealStateModel'.


In [19]:
# client.update_model_version(
#     name=REGISTERED_MODEL_NAME,
#     version=registration.version,|
#     description=f"Best RMSE={best_rmse:.4f} from run {best_run_id}",
# )
# client.transition_model_version_stage(
#     name=REGISTERED_MODEL_NAME,
#     version=registration.version,
#     stage="Production",
# )

# print(
#     f"✅ Best model registered: {REGISTERED_MODEL_NAME} v{registration.version}, "
#     f"RMSE={best_rmse:.4f}, stage=Production"
# )

# # =====================================================
# # 5. Load the Production model
# # =====================================================
# # Option A: load by stage (always gives the latest model in that stage)
# prod_uri = f"models:/{REGISTERED_MODEL_NAME}/Production"
# best_model = mlflow.sklearn.load_model(prod_uri)

In [20]:
# 5. Load a specific version
version_uri = f"models:/{REGISTERED_MODEL_NAME}/{registration.version}"
best_pipeline = mlflow.sklearn.load_model(version_uri)

In [21]:
# Download the artifact to a real local file
local_path = client.download_artifacts(best_run_id, "model/serving_input_example.json")

with open(local_path) as f:
    data = json.load(f)

In [22]:
mlflow.end_run()

In [23]:
best_pipeline.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['diachi', 'quan', 'huyen',
                                                   'loaihinhnhao',
                                                   'giaytophaply', 'duong']),
                                                 ('scale', StandardScaler(),
                                                  ['sotang', 'sophongngu',
                                                   'dientich', 'dai',
                                                   'rong'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, cols...
                              feature_types=None, feature_weights=None,
                              gam

In [24]:
best_pipeline

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('scale', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [25]:
test = pd.DataFrame(columns=data['dataframe_split']['columns'], data=data['dataframe_split']['data'])
test.head(2)

,diachi,quan,huyen,loaihinhnhao,giaytophaply,duong,sotang,sophongngu,dientich,dai,rong,dongia,y_test_pred,perc_diff,flag_perc_diff
0,"Đường Phú Lương, Phường Phú Lương, Quận Hà Đôn...",Quận Hà Đông,Phường Phú Lương,"Nhà ngõ, hẻm",Đã có sổ,Đường Phú Lương,4,3.0,32.0,9.0,3.0,45.31,67.259644,48.443265,0
1,"Đường Phan Văn Trường, Phường Dịch Vọng Hậu, ...",Quận Cầu Giấy,Phường Dịch Vọng Hậu,"Nhà ngõ, hẻm",None,Đường Phan Văn Trường,0,6.0,40.0,NaN,NaN,75.00,106.789482,42.385976,0


In [26]:
print(test['flag_perc_diff'].sum()/len(test) * 100 == best_accuracy)

True
